In [1]:
#웹 크롤링 라이브러리(beautifulsoup4) 설치 
!pip install beautifulsoup4

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
url = 'https://www.naver.com/'
# HTTP GET 요청
response = requests.get(url)
# 응답 상태 코드 확인
if response.status_code == 200:
    print("웹 페이지를 성공적으로 가져왔습니다.")
else:
    print("웹 페이지를 가져오는 데 실패했습니다. 상태 코드:", response.status_code)


웹 페이지를 성공적으로 가져왔습니다.


In [26]:
# 크롤링할 키워드 리스트
keywords = ["청주 맛집 추천"]

# 네이버 블로그 검색 URL 설정
base_url = "https://search.naver.com/search.naver"
params = {
    "where": "post",  # 블로그 검색
    "query": "",      # 검색어 (빈 값으로 초기화)
}

# 키워드별로 블로그 글 가져오기
for keyword in keywords:
    print(f"키워드 '{keyword}'에 대한 블로그 글을 가져옵니다...")
    params["query"] = keyword
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # class="lst_view" 요소 가져오기
        lst_view = soup.find("ul", class_="lst_view")  # div 태그에서 class="lst_view"인 요소 찾기
        
        if lst_view:
            print("lst_view 요소를 성공적으로 가져왔습니다.")
            print(lst_view.prettify())  # 가져온 HTML 구조 출력
        else:
            print("lst_view 요소를 찾을 수 없습니다.")
    else:
        print(f"키워드 '{keyword}'에 대한 데이터를 가져오는 데 실패했습니다. 상태 코드:", response.status_code)

키워드 '청주 맛집 추천'에 대한 블로그 글을 가져옵니다...
lst_view 요소를 성공적으로 가져왔습니다.
<ul class="lst_view">
 <li class="bx">
  <div class="view_wrap">
   <div class="user_box">
    <div class="api_save_group _keep_wrap">
     <a aria-pressed="false" class="btn_save _keep_trigger" data-url="https://blog.naver.com/bylee19/223840085767" href="#" onclick="tCR('a=ugB_bsR*x.kep&amp;r=1&amp;i=90000003_00000000000000341DE8A707&amp;u=javascript'); return false;" role="button">
      <i class="spnew ico_save">
       문서 저장하기
      </i>
     </a>
     <div class="api_ly_save _keep_save_layer">
      <a class="spnew_af item item_save _keep_save" data-cr-off="a=ugB_bsR*x.kepoff&amp;r=1&amp;i=90000003_00000000000000341DE8A707&amp;u=javascript" data-cr-on="a=ugB_bsR*x.kepon&amp;r=1&amp;i=90000003_00000000000000341DE8A707&amp;u=javascript" href="#" role="button">
       Keep에 저장
      </a>
      <a class="spnew_af item item_quick" href="https://keep.naver.com/" onclick="return goOtherCR(this,'a=ugB_bsR*x.kephome&amp;r=1&amp;

In [22]:
text = str(soup.find(class="sc_new sp_nreview _fe_view_root _prs_ugB_bsR"))
text

SyntaxError: invalid syntax (1365484517.py, line 1)

In [ ]:
# HTML 내용 확인
with open("naver_search_result.html", "w", encoding="utf-8") as file:
    file.write(response.text)
    